# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [2]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'C:\BoSSS-Databases\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [4]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,Password,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]",,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\dc1\userspace\kummer\cluster,False,FDY-WindowsHPC,dotnet,,[ \\dc1\userspace\kummer == ],FDY\kummer,DC2,<null>,Normal,True,,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,W:\bosss_deploy,False,LbI,dotnet,,,fk69umer,lcluster8.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,<null>,/home/fk69umer/bosss_deploy,project01287,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\bosss_deploy,False,Lb2-specialPrj,dotnet,,[ X: == /work/scratch/fk69umer ],fk69umer,lcluster16.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",/work/scratch/fk69umer/bosss_deploy,special00006,<null>,False
4,BoSSS.Application.BoSSSpad.SlurmClient,X:\jenkins\deploy,False,Lb2-specialPrj-Jenkins,dotnet,,[ X:\jenkins\databases == /work/scratch/fk69umer/jenkins/databases ],fk69umer,lcluster16.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",/work/scratch/fk69umer/jenkins/deploy,special00006,<null>,False


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [5]:
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]"


In [6]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 18:23:40	60ef7933...
#1: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 17:11:01	ba3ef7e3...
#2: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 16:48:25	7f1469ba...
#3: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 16:35:00	fb3e532c...
#4: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 16:34:05	1dedd133...
#5: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 15:22:16	fbef6e35...
#6: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 12:53:57	d825d868...
#7: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 11:38:49	cdc1164e...
#8: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 11:34:42	0b3d9c62...
#9: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/30/2021 10:12:10	eae39f75...


## Spherical Harmonics

In [9]:
var Case1 = IMatrixExtensions.LoadFromTextFile("surfaceDropCase1.txt");

In [11]:
double[] angle1 = Case1.GetColumn(0);
double[] radius1 = Case1.GetColumn(1);
int I = angle1.Length;
double[] x1 = new double[I], y1 = new double[I];
for( int i = 0; i < I; i++) {
   x1[i] = Math.Sin(angle1[i])*radius1[i];
   y1[i] = Math.Cos(angle1[i])*radius1[i];
}

In [32]:
double case1Func(double angle) {
   double radius = 0.966781 + 0.4*SphericalHarmonicsLogging.MyLegendre(2,0,Math.Cos(angle));
   return radius;
}

In [33]:
double[] x1r = new double[I], y1r = new double[I];
for( int i = 0; i < I; i++) {
   x1r[i] = Math.Sin(angle1[i])*case1Func(angle1[i]);
   y1r[i] = Math.Cos(angle1[i])*case1Func(angle1[i]);
}

In [34]:
Plot(x1, y1, "zrnic", ".xb", x1r, y1r, "kmmr", "- red")

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5 
 
 
 
 
 -1 
 
 
 
 
 -0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 
 
 
 
 zrnic 
 
 
 zrnic 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmmr 
 
 
 kmmr 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M716.2,52.7 L758.4,52.7 M53.9,41.0 L66.2,41.0 L78.5,41.1 L90.8,41.2 L103.1,41.4 L115.4,41.6
 L127.7,41.8 L139.9,42.1 L152.1,42.5 L164.2,42.9 L176.4,43.3 L188.4,43.8 L200.5,44.4 L212.4,44.9
 L224.3,45.6 L236.2,46.2 L248.0,46.9 L259.7,47.7 L271.4,48.5 L282.9,49.3 L294.4,50.2 L305.8,51.1
 L317.1,52.1 L328.4,53.1 L339.5,54.2 L350.5,55.2 L361.4,56.4 L372.2,57.5 L383.0,58.7 L393.5,60.0
 L404.0,61.3 L414.4,62.6 L424.6,63.9 L434.7,65.3 L444.7,66.8 L454.5,68.2 L464.3,69.7 L473.8,71.2
 L483.3,72.8 L492.6,74.4 L501.7,76.0 L510.7,77.6 L519.6,79.3 L528.3,81.0 L536.9,82.7 L545.3,84.5
 L553.5,86.3 L561.6,88.1 L569.5,89.9 L577.3,91.7 L584.9,93.6 L592.4,95.5 L599.7,97.4 L606.8,99.3
 L613.8,101.3 L620.6,103.3 L627.3,105.3 L633.7,107.3 L640.0,109.3 L646.2,111.3 L652.2,113.4 L658.0,115.4
 L663.6,117.5 L669.1,119.6 L674.4,121.7 L679.6,123.8 L684.6,125.9 L689.4,128.0 L694.1,130.1 L698.6,132.2
 L703.0,134.4 L707.2,136.5 L711.2,138.7 L715.1,140.8 L718.8,143.0 L722.4,145.1 L725.8,147.3 L729.1,149.4
 L732.2,151.6 L735.2,153.7 L738.0,155.9 L740.7,158.0 L743.3,160.2 L745.7,162.3 L748.0,164.4 L750.1,166.6
 L752.2,168.7 L754.1,170.8 L755.8,172.9 L757.5,175.0 L759.0,177.1 L760.4,179.2 L761.7,181.3 L762.9,183.3
 L764.0,185.4 L764.9,187.4 L765.8,189.5 L766.6,191.5 L767.3,193.5 L767.8,195.5 L768.3,197.5 L768.7,199.5
 L769.1,201.4 L769.3,203.4 L769.5,205.3 L769.6,207.3 L769.6,209.2 L769.5,211.1 L769.4,213.0 L769.3,214.8
 L769.0,216.7 L768.7,218.5 L768.4,220.4 L768.0,222.2 L767.6,224.0 L767.1,225.7 L766.6,227.5 L766.1,229.3
 L765.5,231.0 L764.9,232.7 L764.3,234.5 L763.6,236.2 L762.9,237.8 L762.3,239.5 L761.5,241.2 L760.8,242.8
 L760.1,244.4 L759.4,246.1 L758.6,247.7 L757.9,249.3 L757.2,250.8 L756.4,252.4 L755.7,254.0 L755.0,255.5
 L754.3,257.0 L753.6,258.6 L752.9,260.1 L752.3,261.6 L751.6,263.1 L751.0,264.6 L750.4,266.0 L749.8,267.5
 L749.3,269.0 L748.8,270.4 L748.3,271.9 L747.8,273.3 L747.4,274.7 L747.0,276.2 L746.6,277.6 L746.3,279.0
 L746.0,280.4 L745.8,281.8 L745.6,283.2 L745.4,284.6 L745.2,286.0 L745.1,287.4 L745.1,288.8 L745.1,290.2
 L745.1,291.6 L745.1,293.0 L745.2,294.4 L745.4,295.8 L745.5,297.2 L745.7,298.6 L746.0,300.1 L746.3,301.5
 L746.6,302.9 L746.9,304.3 L747.3,305.7 L747.7,307.2 L748.2,308.6 L748.7,310.1 L749.2,311.5 L749.7,313.0
 L750.3,314.4 L750.9,315.9 L751.5,317.4 L752.1,318.9 L752.8,320.4 L753.5,321.9 L754.2,323.4 L754.9,324.9
 L755.6,326.5 L756.3,328.0 L757.0,329.6 L757.8,331.2 L758.5,332.8 L759.3,334.4 L760.0,336.0 L760.7,337.6
 L761.4,339.3 L762.1,340.9 L762.8,342.6 L763.5,344.3 L764.2,346.0 L764.8,347.7 L765.4,349.4 L766.0,351.1
 L766.5,352.9 L767.1,354.7 L767.5,356.5 L768.0,358.2 L768.4,360.1 L768.7,361.9 L769.0,363.7 L769.2,365.6
 L769.4,367.4 L769.5,369.3 L769.6,371.2 L769.6,373.1 L769.5,375.1 L769.3,377.0 L769.1,378.9 L768.8,380.9
 L768

## Grid Creation

In [13]:
BoSSSshell.WorkflowMgm.Grids

IOException caught during creation/opening of database: The network path was not found. : '\\dc1\userspace\kummer\OscillatingDroplet3D'.
DirectoryNotFoundException caught during creation/opening of database: Could not find a part of the path 'X:\OscillatingDroplet3D'..
DirectoryNotFoundException caught during creation/opening of database: Could not find a part of the path 'X:\jenkins\databases\OscillatingDroplet3D'..


#0: { Guid = 2ec27b7b-3fa3-453d-b393-d6f6ad4f8eee; Name = OscillatingDroplet3D_12x12x24_quarterDomain; Cell Count = 3456; Dim = 3 }
#1: { Guid = 5623231e-42a5-4842-8a77-90cfe61c382d; Name = OscillatingDroplet3D_6x6x12_quarterDomain; Cell Count = 432; Dim = 3 }


### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [14]:
int[] Resolutions = new int[] { 6, 12 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 0.003, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-0.003, 0.003, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}

Grid already found in database - identifid by name OscillatingDroplet3D_6x6x12_quarterDomain
Grid already found in database - identifid by name OscillatingDroplet3D_12x12x24_quarterDomain


In [15]:
wmg.Grids

#0: { Guid = 2ec27b7b-3fa3-453d-b393-d6f6ad4f8eee; Name = OscillatingDroplet3D_12x12x24_quarterDomain; Cell Count = 3456; Dim = 3 }
#1: { Guid = 5623231e-42a5-4842-8a77-90cfe61c382d; Name = OscillatingDroplet3D_6x6x12_quarterDomain; Cell Count = 432; Dim = 3 }


In [16]:
//wmg.Sessions[0].Delete(true);

## Setup of control objects for a solver runs

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();

In [ ]:
Controls.Clear();
int[] DegreeS = new int[] { 3 };
int[] modeS = new int[] {2, 3, 4};
int[] aP_indiceS = new int[] { 0, 1, 2 };
bool[] useARM = new bool[] { false, true };

foreach(bool bARM in useARM) {
foreach(int k in DegreeS) {
foreach(var grd in Grids.Take(1)) {
foreach(int mode in modeS) {
foreach(int aP_index in aP_indiceS) {
    long J = grd.NumberOfCells;
    string JobName = $"J{J}k{k}_arm{(bARM ? 1 : 0)}_mode{mode}_aP{aP_index}";
    Console.WriteLine("Case: " + JobName);
    double r0 = 0.001;
    var C = Droplet.OscillatingDroplet3D_LegendrePolynomials(r0, mode, aP_index);
    C.SetGrid(grd);
    C.SetDGdegree(3);
    C.SessionName = JobName;
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1260.0;
    C.PhysicalParameters.rho_B = 1.26;
    C.PhysicalParameters.mu_A = 0.0094;
    C.PhysicalParameters.mu_B = 9.4E-06;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 0.007;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.dtFixed = 5E-05;
    C.NoOfTimesteps = 1000;
    
    if(bARM) {
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = 1 }
        );
    }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging());
    
    Controls.Add(C);
    
}
}
}
}
}

In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal");
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal");
    }
}

## Launch Jobs

In [ ]:
Controls.Count

In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

In [ ]:
wmg.AllJobs

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*5), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()